# Urban Sprawl Analysis

This notebook demonstrates how to use the Urban Sprawl Analysis toolkit to analyze land cover change and urban expansion using Google Earth Engine and Python.

In [ ]:
# Setup: Import modules and configure environment
import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), 'src'))
from gee_utils import GEEProcessor
from land_cover import LandCoverClassifier
from change_detection import ChangeDetector
from visualization import UrbanSprawlVisualizer

# Initialize processors
gee_processor = GEEProcessor()
classifier = LandCoverClassifier(gee_processor)
change_detector = ChangeDetector(gee_processor)
visualizer = UrbanSprawlVisualizer()

## 1. Define Study Area and Time Periods
(Edit `config/settings.py` to change these parameters.)

In [ ]:
from settings import STUDY_AREA, TIME_PERIODS
print('Study Area:', STUDY_AREA['name'])
print('Time Period:', TIME_PERIODS['start_year'], '-', TIME_PERIODS['end_year'])

## 2. Download and Preprocess Satellite Imagery
Get Landsat collections for the start and end years.

In [ ]:
start_year = TIME_PERIODS['start_year']
end_year = TIME_PERIODS['end_year']

# Get image collections
collection1 = gee_processor.get_landsat_collection(f'{start_year}-01-01', f'{start_year}-12-31')
collection2 = gee_processor.get_landsat_collection(f'{end_year}-01-01', f'{end_year}-12-31')

# Create composite images
image1 = gee_processor.get_composite_image(collection1)
image2 = gee_processor.get_composite_image(collection2)

## 3. Land Cover Classification
Classify land cover for both years.

In [ ]:
classified1 = classifier.classify_land_cover(image1)
classified2 = classifier.classify_land_cover(image2)

## 4. Change Detection
Analyze changes in land cover and urban expansion.

In [ ]:
change_image = change_detector.detect_land_cover_changes(classified1, classified2)
veg_changes = change_detector.analyze_vegetation_changes(image1, image2)
urban_changes = change_detector.analyze_urban_expansion(image1, image2)
print('Vegetation Changes:', veg_changes)
print('Urban Expansion:', urban_changes)

## 5. Visualization
Create and display maps and charts.

In [ ]:
# Interactive map for classified land cover
m1 = visualizer.create_interactive_map(classified1, title=f'Land Cover {start_year}')
m2 = visualizer.create_interactive_map(classified2, title=f'Land Cover {end_year}')
m1.save('results/maps/land_cover_start.html')
m2.save('results/maps/land_cover_end.html')

# Land cover statistics
stats1 = classifier.get_class_statistics(classified1)
stats2 = classifier.get_class_statistics(classified2)

fig1 = visualizer.create_land_cover_chart(stats1, title=f'Land Cover {start_year}')
fig2 = visualizer.create_land_cover_chart(stats2, title=f'Land Cover {end_year}')
visualizer.save_chart(fig1, f'land_cover_{start_year}')
visualizer.save_chart(fig2, f'land_cover_{end_year}')

## 6. Export Results
Save metrics and change analysis results.

In [ ]:
visualizer.save_metrics_csv(veg_changes, 'vegetation_changes')
visualizer.save_metrics_csv(urban_changes, 'urban_expansion')